In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [3]:
xl = pd.ExcelFile('./NVT_Metadatentabelle_MASTER.xlsm') ## load Excel File to display sheet_names

In [190]:
xl.sheet_names  ## display sheet_names

['Produktionen',
 'Ereignisse',
 'Objekte VIDEOS',
 'Objekte TEXT',
 'Objekte BILD',
 'Objekte AUDIO',
 '||_Personen',
 '||_Gruppen_Ensembles',
 '||_Veranstaltungsort',
 '||_Städte',
 '||_Länder',
 'Sammlungen',
 'Serie',
 'TYPEN und GLOSSARE',
 'Fragen und Dok',
 'TODOs',
 'Legende',
 'Unterobjekt Beispiele',
 'Übersicht']

In [272]:
def merge_columns(df, col1, col2, result=None):
    if result == "season":
        col3 = "Spielzeit"
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "season_" + df["ID"] + "_" + df[col1].apply(str).str[:4] + "_" + pr_clean[col2].apply(str).str[:4]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "season_" + df["ID"] + "_" + df[col1].apply(str).str[:4]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "season_" + df["ID"] + "_" + df[col2].apply(str).str[:4]
        return df
    elif result == "description":
        col3 = "BeschreibungQuelle"
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "Beschreibung: " + pr_clean[col1] + " Quelle: " + pr_clean[col2]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "Beschreibung: " + pr_clean[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = " Quelle: " + pr_clean[col2]
        return df
    elif result == "texts":
        col3 = "Texte_Autoren"
        df.loc[df[col1].notna() & df[col2].notna(), col3] = "Text(e): " + pr_clean[col1] + " Autor(en): " + pr_clean[col2]
        df.loc[df[col1].notna() & df[col2].isna(), col3] = "Text(e): " + pr_clean[col1]
        df.loc[df[col1].isna() & df[col2].notna(), col3] = "Autor(en): " + pr_clean[col2]
        return df
    else:
        print("Wrong Keyword, nothing happened")
        return df

In [355]:
def supersplit(df):
    df = df.str.split(";", expand = False) 
    return df

In [265]:
def peoplereplace(df):
    df.str.replace("_","-")
    df.str.replace(",", " ")
    df.str.replace(";", "; ")
    return df

In [393]:
def refreplace(df):
    df = df.str.lower()
    df = df.str.replace("ä", "ae")
    df = df.str.replace("ö","oe")
    df = df.str.replace("ü","ue")
    df = df.str.replace("ß","ss")
    df = df.str.replace("!?,\.", "")
    df = df.str.replace("[^a-zA-Z0-9;]", "_")
    df = df.str.replace("__", "_")
    df = df.str.strip("_")
    return df

In [182]:
## events = pd.read_excel('./NVT_Metadatentabelle_MASTER.xlsm', sheet_name='Ereignisse')

In [201]:
productions = pd.read_excel('./NVT_Metadatentabelle_MASTER.xlsm', sheet_name='Produktionen', header=1) ## load productions sheet with header in row 1 (row 0 is ignored i think)

In [338]:
pr_clean = (productions[productions['Identifier / geeinigter Name'].str.contains('PR', na=False)] ## drops all rows that don't start with 'PR' or are NaN
           .drop(['Musik (für GEMA)'], axis='columns') ## drops unmodeled column
           .drop([2]) ## drops example row
           .reset_index(drop=True) ## resets index to be continouus
           .rename(columns={'Identifier / geeinigter Name':'ID', ## renames columns for future reference
                                          'Produktionsname / Titel':'PR_Titel',
                                          'Quelle (Beschreibung)':'Q_Beschreibung',
                                          'Verwendete Texte':'Verwendete_Texte',
                                          'Autor(en) der Texte':'Autoren_Texte',
                                          'Beteiligte Gruppen / Compagnies':'Beteiligte_Gruppen',
                                          'Sprecher*in':'SprecherIn',
                                          'Darsteller allgem.':'Darsteller',
                                          'Weitere Mitwirkende':'Mitwirkende',
                                          'Spielzeit / Laufzeit Start':'Spielzeit_Start',
                                          'Spielzeit / Laufzeit Ende':'Spielzeit_Ende'
                           })
           )

In [339]:
# pr_clean = merge_season(pr_clean, "Spielzeit_Start", "Spielzeit_Ende", "Spielzeit")

In [340]:
pr_clean = merge_columns(pr_clean, "Spielzeit_Start", "Spielzeit_Ende", result = "season")

In [341]:
pr_clean = merge_columns(pr_clean, "Beschreibung", "Q_Beschreibung", result = "description")

In [397]:
pr_clean = merge_columns(pr_clean, "Verwendete_Texte", "Autoren_Texte", result = "texts")
## in Zukunft müssten die Autoren mit der Personentabelle abgeglichen werden, am besten trotzdem noch als literal

In [366]:
pr_clean.columns

Index(['ID', 'PR_Titel', 'Beschreibung', 'Q_Beschreibung', 'Verwendete_Texte',
       'Autoren_Texte', 'Produktionstyp', 'Beteiligte_Gruppen', 'Konzept',
       'Textbearbeitung', 'Übersetzung', 'Regie', 'SprecherIn',
       'Choreographie', 'Dramaturgie', 'Bühnenbild', 'Kostümdesgin',
       'Figurenbau', 'Maskenbau', 'Lichtdesign', 'Videodesign', 'Schauspieler',
       'Tänzer', 'Darsteller', 'Musiker', 'Komposition', 'Mitwirkende',
       'Premierendatum', 'Dernierendatum', 'Spielzeit_Start', 'Spielzeit_Ende',
       'Veranstaltungsort', 'Stadt', 'Land', 'Spielzeit', 'BeschreibungQuelle',
       'Texte_Autoren'],
      dtype='object')

In [344]:
for idx, i in enumerate(pr_clean.columns):
    if idx == 0:
        print("Referenzliste zum splitten und replacen; I=ID R=reference L=literal N=None\n\nrefreplace für I und R, split für R, erst split dann refreplace\n")
        print(["I", idx, i])
    elif idx in range(6, 27) or idx in range(31, 35):
        print(["R", idx, i])
    elif idx in range(2, 6):
        print(["N", idx])
    else:
        print(["L", idx, i])

Referenzliste zum splitten und replacen; I=ID R=reference L=literal N=None

refreplace für I und R, split für R, erst split dann refreplace

['I', 0, 'ID']
['L', 1, 'PR_Titel']
['N', 2]
['N', 3]
['N', 4]
['N', 5]
['R', 6, 'Produktionstyp']
['R', 7, 'Beteiligte_Gruppen']
['R', 8, 'Konzept']
['R', 9, 'Textbearbeitung']
['R', 10, 'Übersetzung']
['R', 11, 'Regie']
['R', 12, 'SprecherIn']
['R', 13, 'Choreographie']
['R', 14, 'Dramaturgie']
['R', 15, 'Bühnenbild']
['R', 16, 'Kostümdesgin']
['R', 17, 'Figurenbau']
['R', 18, 'Maskenbau']
['R', 19, 'Lichtdesign']
['R', 20, 'Videodesign']
['R', 21, 'Schauspieler']
['R', 22, 'Tänzer']
['R', 23, 'Darsteller']
['R', 24, 'Musiker']
['R', 25, 'Komposition']
['R', 26, 'Mitwirkende']
['L', 27, 'Premierendatum']
['L', 28, 'Dernierendatum']
['L', 29, 'Spielzeit_Start']
['L', 30, 'Spielzeit_Ende']
['R', 31, 'Veranstaltungsort']
['R', 32, 'Stadt']
['R', 33, 'Land']
['R', 34, 'Spielzeit']
['L', 35, 'BeschreibungQuelle']
['L', 36, 'Texte_Autoren']


In [395]:
pr_test = pr_clean.iloc[:, [0,11]].apply(refreplace, axis=1)
## refreplace zwei mal mit splice, weil alle refs beieinander liegen, ansonsten könnte man auch mit
## [:, [0,3,4,12]] einzelne Spalten anwählen
pr_test

,ID,Regie
0,pr_12_traeume_des_dr_fridkin,NaN
1,pr_amapola,ernstgeorg_hering;helmut_strassburger;eberhard...
2,pr_ameriques,NaN
3,pr_aufstand_der_traeume,andreas_ittner;harald_seime
4,pr_august_august,peter_baumgart
5,pr_avat_theater,fethi_osmane;said_benyoucef
6,pr_baustelle,peter_honegger
7,pr_beauti_fools,christian_mattis
8,pr_berliner_ballonfieber_1803,burkhart_seidemann
9,pr_bilder_einer_ausstellung,NaN


In [396]:
pr_test = pr_test.apply(supersplit, axis=1)
pr_test[0:2]

,ID,Regie
0,[pr_12_traeume_des_dr_fridkin],NaN
1,[pr_amapola],"[ernstgeorg_hering, helmut_strassburger, eberh..."
